# Cell–cell communication between Fibro and Hepato subtypes (CellPhoneDB + NicheNet)
related to Figure.1

- **CellPhoneDB (statistical analysis, method 2)**: ligand–receptor interaction significance and interaction scores across **subtype-level** cell groups.
- **NicheNet**: prioritization of upstream ligands from **Fibro subtypes** that may regulate **Hepato subtype** marker programs.

**Important constraint:** there is **no condition contrast** in this analysis. All samples are considered **tumor**.


## Purpose and scope

**Goal.** Quantify and prioritize putative communication **between two lineages**:

- Fibro lineage: `1_Fibro_counts.h5ad`
- Hepato lineage: `1_Hepato_counts.h5ad`

Both inputs are assumed to contain subtype annotations in `obs['subtype']`.

**Key design choices**

1. **No condition-based DE** (e.g. tumor vs normal).  
2. For NicheNet, we define the receiver *gene set of interest* as **Hepato subtype markers vs other Hepato subtypes** (within tumor-only data).
3. Cell types are encoded as `celltype_lr = <Lineage>__<Subtype>` to avoid naming collisions across lineages.


In [ ]:
## =========================
## CONFIG (paths + key params)
## =========================

# ---- Inputs (tumor-only) ----
FIBRO_H5AD  = "1_Fibro_counts.h5ad"
HEPATO_H5AD = "1_Hepato_counts.h5ad"

# ---- Output folders ----
OUTDIR            = "./cell_commu_out"
CPDB_IO_DIR       = f"{OUTDIR}/cpdb_io"
CPDB_OUT_DIR      = f"{OUTDIR}/cpdb_results/fibro_vs_hepato"
NICHENET_IO_DIR   = f"{OUTDIR}/nichenet_io"
NICHENET_OUT_DIR  = f"{OUTDIR}/nichenet_results/fibro_to_hepato"

# ---- CellPhoneDB database (zip) ----
# Example (repo-managed): ./cpdb_db/v5.0.0/cellphonedb.zip
# You must provide a valid CellPhoneDB database zip for your environment.
CPDB_ZIP = "./cpdb_db/v5.0.0/cellphonedb.zip"

# CellPhoneDB parameters
CPDB_ITERATIONS = 1000
CPDB_THRESHOLD  = 0.1
CPDB_PVALUE     = 0.05
CPDB_THREADS    = 8
CPDB_SEED       = 42

# Expression preprocessing (for both CPDB and NicheNet)
TARGET_SUM = 1e4  # normalize_total target sum
LOG1P_BASE = None # None -> natural log; set to 2 for log2(1+x) if needed

# NicheNet parameters (tumor-only, subtype-marker gene set)
NICHENET_EXP_PCT   = 0.05   # expression fraction to call a gene "expressed"
NICHENET_DE_FDR    = 0.05
NICHENET_DE_LOGFC  = 0.25
NICHENET_TOP_LIGS  = 30

# Convenience: celltype label used in both tools
CELLTYPE_KEY = "celltype_lr"   # constructed as <Lineage>__<Subtype>
LINEAGE_KEY  = "lineage"
SUBTYPE_KEY  = "subtype"


In [ ]:
## =========================
## Reproducibility: seeds + packages + versions
## =========================
import os, sys, json, shutil, subprocess, textwrap
from pathlib import Path

import numpy as np
import pandas as pd

np.random.seed(CPDB_SEED)

# Core single-cell I/O
import scanpy as sc
import anndata as ad
from scipy import sparse

print("Python:", sys.version)
print("scanpy:", sc.__version__)
print("anndata:", ad.__version__)
try:
    import cellphonedb
    print("cellphonedb:", cellphonedb.__version__)
except Exception as e:
    print("cellphonedb: not available (install required).", repr(e))


## Data loading and preprocessing


In [ ]:
## =========================
## Load Fibro + Hepato h5ad and build a combined log-normalized object
## =========================

Path(OUTDIR).mkdir(parents=True, exist_ok=True)
Path(CPDB_IO_DIR).mkdir(parents=True, exist_ok=True)
Path(CPDB_OUT_DIR).mkdir(parents=True, exist_ok=True)
Path(NICHENET_IO_DIR).mkdir(parents=True, exist_ok=True)
Path(NICHENET_OUT_DIR).mkdir(parents=True, exist_ok=True)

fibro_raw  = sc.read_h5ad(FIBRO_H5AD)
hepato_raw = sc.read_h5ad(HEPATO_H5AD)

# ---- Basic sanity checks ----
for name, adata in [("Fibro", fibro_raw), ("Hepato", hepato_raw)]:
    if SUBTYPE_KEY not in adata.obs.columns:
        raise KeyError(f"{name}: obs['{SUBTYPE_KEY}'] not found. Please add subtype annotations.")

# ---- Make cell IDs unique across lineages ----
fibro_raw = fibro_raw.copy()
hepato_raw = hepato_raw.copy()
fibro_raw.obs_names  = [f"Fibro_{x}" for x in fibro_raw.obs_names.astype(str)]
hepato_raw.obs_names = [f"Hepato_{x}" for x in hepato_raw.obs_names.astype(str)]

# ---- Add lineage metadata ----
fibro_raw.obs[LINEAGE_KEY]  = "Fibro"
hepato_raw.obs[LINEAGE_KEY] = "Hepato"

def _ensure_counts_layer(adata: ad.AnnData, counts_layer: str = "counts") -> ad.AnnData:
    """Ensure raw counts are available in layers[counts_layer].
    If the layer is missing, we assume adata.X contains counts and store it.
    """
    adata = adata.copy()
    if counts_layer not in adata.layers:
        adata.layers[counts_layer] = adata.X.copy()
    return adata

def make_lognorm(adata: ad.AnnData,
                 counts_layer: str = "counts",
                 target_sum: float = 1e4,
                 log1p_base = None) -> ad.AnnData:
    """Return a copy with log-normalized expression in .X (based on layers[counts_layer])."""
    adata = _ensure_counts_layer(adata, counts_layer=counts_layer)
    adata = adata.copy()
    adata.X = adata.layers[counts_layer].copy()
    sc.pp.normalize_total(adata, target_sum=target_sum)
    sc.pp.log1p(adata, base=log1p_base)
    return adata

fibro = make_lognorm(fibro_raw, counts_layer="counts", target_sum=TARGET_SUM, log1p_base=LOG1P_BASE)
hepato = make_lognorm(hepato_raw, counts_layer="counts", target_sum=TARGET_SUM, log1p_base=LOG1P_BASE)

# ---- Construct a unified celltype label for ligand–receptor analysis ----
fibro.obs[CELLTYPE_KEY]  = fibro.obs[LINEAGE_KEY].astype(str)  + "__" + fibro.obs[SUBTYPE_KEY].astype(str)
hepato.obs[CELLTYPE_KEY] = hepato.obs[LINEAGE_KEY].astype(str) + "__" + hepato.obs[SUBTYPE_KEY].astype(str)

# ---- Merge (inner-join genes to ensure shared feature space) ----
adata_lr = ad.concat([fibro, hepato], join="inner", axis=0, merge="same")
adata_lr.var_names_make_unique()

print(adata_lr)
print("Lineages:", adata_lr.obs[LINEAGE_KEY].value_counts().to_dict())
print("N celltypes:", adata_lr.obs[CELLTYPE_KEY].nunique())

# Save for downstream tools (CPDB + NicheNet)
combined_h5ad = Path(NICHENET_IO_DIR) / "Fibro_Hepato_lognorm.h5ad"
adata_lr.write(combined_h5ad)
print("Wrote:", combined_h5ad)


## CellPhoneDB 


In [ ]:
## =========================
## 1) Prepare CellPhoneDB inputs (meta + normalized counts)
## =========================

# Detect gene ID type for CellPhoneDB
def guess_counts_data(var_names) -> str:
    var_names = list(map(str, var_names[: min(2000, len(var_names))]))
    frac_ensg = np.mean([v.startswith("ENSG") for v in var_names])
    return "ensembl" if frac_ensg > 0.5 else "hgnc_symbol"

COUNTS_DATA = guess_counts_data(adata_lr.var_names)
print("COUNTS_DATA:", COUNTS_DATA)

# Meta file: Cell and cell_type
meta_df = pd.DataFrame({
    "Cell": adata_lr.obs_names.astype(str),
    "cell_type": adata_lr.obs[CELLTYPE_KEY].astype(str).values,
})
meta_file_path = str(Path(CPDB_IO_DIR) / "metadata.tsv")
meta_df.to_csv(meta_file_path, sep="\t", index=False)
print("Wrote:", meta_file_path, meta_df.shape)

# Normalized counts (h5ad): ensure .X is log-normalized
counts_h5ad_path = str(Path(CPDB_IO_DIR) / "normalised_counts.h5ad")
adata_cpdb = adata_lr.copy()
adata_cpdb.write(counts_h5ad_path)
print("Wrote:", counts_h5ad_path)

# Optional microenvironment file: put all cell types into a single environment
microenv_path = str(Path(CPDB_IO_DIR) / "microenvironment.tsv")
pd.DataFrame({
    "cell_type": sorted(meta_df["cell_type"].unique()),
    "microenvironment": "Env1",
}).to_csv(microenv_path, sep="\t", index=False)

## =========================
## 2) Run CellPhoneDB statistical analysis (method 2)
## =========================
# NOTE: This step can be slow (iterations = CPDB_ITERATIONS).
# It will be skipped automatically if outputs already exist.

means_files = list(Path(CPDB_OUT_DIR).glob("statistical_analysis_means*.txt"))
if len(means_files) > 0:
    print("Existing CellPhoneDB results detected -> skip run.")
else:
    if not Path(CPDB_ZIP).exists():
        raise FileNotFoundError(
            f"CellPhoneDB database zip not found: {CPDB_ZIP}\n"
            "Please download / point CPDB_ZIP to a valid cellphonedb.zip."
        )

    try:
        from cellphonedb.src.core.methods import cpdb_statistical_analysis_method
    except Exception as e:
        raise ImportError(
            "cellphonedb is not installed. Install it (e.g. pip install cellphonedb) "
            "and ensure the DB zip is available."
        ) from e

    cpdb_statistical_analysis_method.call(
        cpdb_file_path     = CPDB_ZIP,
        meta_file_path     = meta_file_path,
        counts_file_path   = counts_h5ad_path,   # we pass a h5ad with .X = log-normalized expression
        counts_data        = COUNTS_DATA,        # 'hgnc_symbol' or 'ensembl'
        score_interactions = True,
        iterations         = CPDB_ITERATIONS,
        threshold          = CPDB_THRESHOLD,
        threads            = CPDB_THREADS,
        debug_seed         = CPDB_SEED,
        result_precision   = 3,
        pvalue             = CPDB_PVALUE,
        separator          = "|",
        output_path        = CPDB_OUT_DIR,
    )
    print("CellPhoneDB finished. Output:", CPDB_OUT_DIR)


### Load and summarize CellPhoneDB outputs

In [ ]:
## =========================
## Load CellPhoneDB results and focus on Fibro ↔ Hepato subtype pairs
## =========================
from glob import glob

def load_cpdb_results(result_dir: str, prefix: str = "statistical_analysis") -> dict:
    """Load CellPhoneDB output tables from a result directory."""
    result_dir = str(result_dir)
    results = {}
    file_map = {
        "means": f"{prefix}_means",
        "pvalues": f"{prefix}_pvalues",
        "significant_means": f"{prefix}_significant_means",
        "interaction_scores": f"{prefix}_interaction_scores",
        "deconvoluted": f"{prefix}_deconvoluted",
        "deconvoluted_percents": f"{prefix}_deconvoluted_percents",
    }
    for key, base in file_map.items():
        matches = [f for f in os.listdir(result_dir) if f.startswith(base) and f.endswith(".txt")]
        if len(matches) == 0:
            continue
        path = os.path.join(result_dir, matches[0])
        results[key] = pd.read_csv(path, sep="\t")
        print(f"Loaded {key}: {results[key].shape} -> {path}")
    return results

cpdb = load_cpdb_results(CPDB_OUT_DIR)

if "means" not in cpdb or "pvalues" not in cpdb:
    raise RuntimeError("Missing CellPhoneDB output tables (means/pvalues). Did the CPDB run finish successfully?")

means_df = cpdb["means"]
pvals_df = cpdb["pvalues"]

# Cell-pair columns are those containing the CPDB separator (default '|')
pair_cols = [c for c in pvals_df.columns if isinstance(c, str) and "|" in c]

def is_fibro_hepato_pair(col: str) -> bool:
    a, b = col.split("|")
    return ((a.startswith("Fibro__") and b.startswith("Hepato__")) or
            (a.startswith("Hepato__") and b.startswith("Fibro__")))

fh_cols = [c for c in pair_cols if is_fibro_hepato_pair(c)]
print("Fibro↔Hepato pair columns:", len(fh_cols))

# Significance mask: p < CPDB_PVALUE and mean > 0
# NOTE: 'means' and 'pvalues' should have the same pair columns
sig = (pvals_df[fh_cols] < CPDB_PVALUE) & (means_df[fh_cols] > 0)

# Count significant interactions per pair column
sig_counts = sig.sum(axis=0).rename("n_sig_interactions").reset_index().rename(columns={"index": "pair"})
sig_counts[["celltype_a", "celltype_b"]] = sig_counts["pair"].str.split("|", expand=True)

# Build a Fibro (rows) x Hepato (cols) matrix of interaction counts
def split_pair(a, b):
    if a.startswith("Fibro__") and b.startswith("Hepato__"):
        return a, b
    if a.startswith("Hepato__") and b.startswith("Fibro__"):
        return b, a
    return None, None

pairs = sig_counts.apply(lambda r: split_pair(r["celltype_a"], r["celltype_b"]), axis=1, result_type="expand")
sig_counts["fibro_subtype"]  = pairs[0]
sig_counts["hepato_subtype"] = pairs[1]

mat = sig_counts.pivot_table(index="fibro_subtype", columns="hepato_subtype", values="n_sig_interactions", fill_value=0)

summary_dir = Path(OUTDIR) / "cpdb_results_summary"
summary_dir.mkdir(parents=True, exist_ok=True)

sig_counts_path = summary_dir / "cpdb_fibro_hepato_sig_counts.tsv"
mat_path        = summary_dir / "cpdb_fibro_hepato_sig_matrix.tsv"

sig_counts.to_csv(sig_counts_path, sep="\t", index=False)
mat.to_csv(mat_path, sep="\t")

print("Wrote:", sig_counts_path)
print("Wrote:", mat_path)

# Display top pairs
sig_counts.sort_values("n_sig_interactions", ascending=False).head(15)


### Optional: visualize a specific Fibro ↔ Hepato pair 

In [ ]:
## =========================
## Optional dotplot for a selected Fibro–Hepato pair
## =========================
celltypes = sorted(adata_lr.obs[CELLTYPE_KEY].unique())
print("Available celltypes (first 30):")
print(celltypes[:30])

CELLTYPE1 = None  # e.g. "Fibro__ecmFibro_FAP"
CELLTYPE2 = None  # e.g. "Hepato__Malignant_C2"

if CELLTYPE1 is not None and CELLTYPE2 is not None:
    try:
        import ktplotspy as kpy
        p = kpy.plot_cpdb(
            adata=adata_lr,
            cell_type1=CELLTYPE1,
            cell_type2=CELLTYPE2,
            means=cpdb["means"],
            pvals=cpdb["pvalues"],
            celltype_key=CELLTYPE_KEY,
            figsize=(10, 18),
            title=f"{CELLTYPE1} ↔ {CELLTYPE2} (CellPhoneDB)",
        )
        p
    except Exception as e:
        print("ktplotspy is not available or failed to plot:", repr(e))
else:
    print("Set CELLTYPE1 and CELLTYPE2 to generate a dotplot.")


## NicheNet (Fibro → Hepato, tumor-only)

- **Receiver**: each `Hepato__<Subtype>` (one at a time)
- **Gene set of interest (geneset_oi)**: subtype markers of that receiver vs other Hepato subtypes (within tumor-only cells)
- **Sender pool**: all `Fibro__<Subtype>` cell types

This section is implemented in **R** (NicheNet is an R package). The notebook writes an R script that:
1) reads the combined `.h5ad` produced above  
2) runs NicheNet ligand activity inference per Hepato subtype  
3) writes results tables to `NICHENET_OUT_DIR`

You can execute it either by:
- running the R script with `Rscript`, or
- copying the R section into an R kernel notebook.


In [ ]:
## =========================
## Write an R script for NicheNet (Fibro → Hepato, tumor-only)
## =========================

r_script_path = Path(NICHENET_IO_DIR) / "run_nichenet_fibro_to_hepato.R"

r_code = f'''
suppressPackageStartupMessages({{
  library(dplyr)
  library(tidyr)
  library(ggplot2)
  library(Seurat)
  library(nichenetr)
  library(SingleCellExperiment)
  library(zellkonverter)
}})

# -------------------------
# CONFIG
# -------------------------
combined_h5ad <- "{str(combined_h5ad)}"
outdir <- "{NICHENET_OUT_DIR}"

exp_pct   <- {NICHENET_EXP_PCT}
de_fdr    <- {NICHENET_DE_FDR}
de_logfc  <- {NICHENET_DE_LOGFC}
top_ligs  <- {NICHENET_TOP_LIGS}

dir.create(outdir, recursive = TRUE, showWarnings = FALSE)

# -------------------------
# Load data (combined h5ad)
# -------------------------
sce <- readH5AD(combined_h5ad)
assay_names <- SummarizedExperiment::assayNames(sce)
message("Assays in SCE: ", paste(assay_names, collapse = ", "))

# Use the first assay as expression if 'X' is absent
expr_assay <- if ("X" %in% assay_names) "X" else assay_names[1]
message("Using assay for Seurat conversion: ", expr_assay)

obj <- as.Seurat(sce, counts = expr_assay, data = expr_assay)

# Metadata keys (must exist; created in the Python part)
stopifnot(all(c("{LINEAGE_KEY}", "{SUBTYPE_KEY}", "{CELLTYPE_KEY}") %in% colnames(obj@meta.data)))

Idents(obj) <- "{CELLTYPE_KEY}"

sender_celltypes <- sort(unique(obj@meta.data${CELLTYPE_KEY}[obj@meta.data${LINEAGE_KEY} == "Fibro"]))
receiver_celltypes <- sort(unique(obj@meta.data${CELLTYPE_KEY}[obj@meta.data${LINEAGE_KEY} == "Hepato"]))

message("N sender celltypes (Fibro): ", length(sender_celltypes))
message("N receiver celltypes (Hepato): ", length(receiver_celltypes))

# -------------------------
# Load NicheNet prior data
# -------------------------
data("lr_network", package = "nichenetr")
data("ligand_target_matrix", package = "nichenetr")

# Ensure ligand_target_matrix orientation: targets in rows, ligands in columns
ltm <- as.matrix(ligand_target_matrix)

# -------------------------
# Helper: receiver subtype markers (no condition contrast)
# -------------------------
obj_hep <- subset(obj, subset = {LINEAGE_KEY} == "Hepato")
Idents(obj_hep) <- "{CELLTYPE_KEY}"

get_receiver_markers <- function(receiver_label) {{
  de <- FindMarkers(
    object = obj_hep,
    ident.1 = receiver_label,
    ident.2 = NULL,
    min.pct = 0.10,
    logfc.threshold = 0.0,
    test.use = "wilcox"
  )
  de$gene <- rownames(de)

  # Harmonize logFC column name across Seurat versions
  if ("avg_log2FC" %in% colnames(de)) {{
    de$logFC <- de$avg_log2FC
  }} else if ("avg_logFC" %in% colnames(de)) {{
    de$logFC <- de$avg_logFC
  }} else {{
    stop("Cannot find avg_log2FC or avg_logFC in DE table.")
  }}

  de <- de %>%
    filter(p_val_adj <= de_fdr, logFC >= de_logfc) %>%
    arrange(desc(logFC))

  return(de)
}}

# -------------------------
# Main loop: Fibro -> each Hepato subtype
# -------------------------
all_results <- list()

for (receiver in receiver_celltypes) {{

  message("---- Receiver: ", receiver, " ----")

  # Expressed genes in receiver
  expressed_genes_receiver <- get_expressed_genes(receiver, obj, pct = exp_pct)

  # Expressed receptors in receiver
  receptors <- unique(lr_network$to)
  expressed_receptors <- intersect(receptors, expressed_genes_receiver)

  # Expressed genes across sender cell types (union)
  expressed_sender_union <- sender_celltypes %>%
    lapply(function(s) get_expressed_genes(s, obj, pct = exp_pct)) %>%
    unlist(use.names = FALSE) %>%
    unique()

  ligands <- unique(lr_network$from)
  expressed_ligands <- intersect(ligands, expressed_sender_union)

  # Potential ligands (sender ligands with receptors expressed in receiver)
  potential_ligands <- lr_network %>%
    filter(from %in% expressed_ligands, to %in% expressed_receptors) %>%
    pull(from) %>%
    unique()

  # Receiver gene set of interest = receiver subtype markers (no condition contrast)
  de_tbl <- get_receiver_markers(receiver)
  geneset_oi <- de_tbl$gene

  # Background genes must be expressed and part of the target universe
  targets_universe <- rownames(ltm)
  background_expressed_genes <- intersect(expressed_genes_receiver, targets_universe)

  # Ensure geneset_oi is inside target universe
  geneset_oi <- intersect(geneset_oi, targets_universe)

  # Ensure potential ligands exist in ligand-target matrix (ligands typically in columns)
  lig_in_cols <- mean(potential_ligands %in% colnames(ltm))
  lig_in_rows <- mean(potential_ligands %in% rownames(ltm))
  if (lig_in_rows > lig_in_cols) {{
    message("Transposing ligand_target_matrix to match (targets in rows, ligands in cols).")
    ltm <- t(ltm)
  }}
  potential_ligands <- intersect(potential_ligands, colnames(ltm))

  if (length(geneset_oi) < 10) {{
    message("Skip receiver (too few geneset_oi after filtering): ", length(geneset_oi))
    next
  }}
  if (length(potential_ligands) < 10) {{
    message("Skip receiver (too few potential ligands): ", length(potential_ligands))
    next
  }}

  ligand_activities <- predict_ligand_activities(
    geneset = geneset_oi,
    background_expressed_genes = background_expressed_genes,
    ligand_target_matrix = ltm,
    potential_ligands = potential_ligands
  ) %>%
    arrange(desc(aupr_corrected)) %>%
    mutate(rank = rank(desc(aupr_corrected)))

  top_ligands <- ligand_activities %>% head(top_ligs) %>% pull(test_ligand) %>% unique()

  # Save per-receiver outputs
  safe_receiver <- gsub("[^A-Za-z0-9_]+", "_", receiver)
  out_prefix <- file.path(outdir, paste0("receiver_", safe_receiver))

  write.table(ligand_activities, paste0(out_prefix, "_ligand_activities.tsv"),
              sep="\t", row.names=FALSE, quote=FALSE)

  write.table(de_tbl, paste0(out_prefix, "_receiver_markers.tsv"),
              sep="\t", row.names=FALSE, quote=FALSE)

  # Ligand expression across Fibro sender subtypes
  obj_fib <- subset(obj, subset = {LINEAGE_KEY} == "Fibro")
  Idents(obj_fib) <- "{CELLTYPE_KEY}"
  avg_expr <- AverageExpression(obj_fib, features = top_ligands, slot = "data")$RNA
  write.table(avg_expr, paste0(out_prefix, "_avgexpr_top_ligands_in_fibro.tsv"),
              sep="\t", quote=FALSE)

  # Ligand-receptor links consistent with expressed receptors
  lr_links <- lr_network %>%
    filter(from %in% top_ligands, to %in% expressed_receptors) %>%
    distinct(from, to)

  write.table(lr_links, paste0(out_prefix, "_lr_links.tsv"),
              sep="\t", row.names=FALSE, quote=FALSE)

  all_results[[receiver]] <- ligand_activities %>% mutate(receiver = receiver)
}}

# Combined ligand activity table across receivers
if (length(all_results) > 0) {{
  all_tbl <- bind_rows(all_results)
  write.table(all_tbl, file.path(outdir, "ALL_receivers_ligand_activities.tsv"),
              sep="\t", row.names=FALSE, quote=FALSE)
}}

message("NicheNet finished. Results in: ", outdir)
'''

r_script_path.write_text(r_code)
print("Wrote R script:", r_script_path)

# Optional: run if Rscript exists
if shutil.which("Rscript") is None:
    print("Rscript not found. Run the script manually in an R environment:")
    print(f"  Rscript {r_script_path}")
else:
    cmd = ["Rscript", str(r_script_path)]
    print("Running:", " ".join(cmd))
    subprocess.run(cmd, check=True)


## Outputs

This notebook produces:

### CellPhoneDB
- `CPDB_OUT_DIR/`  
  CellPhoneDB output tables (`means`, `pvalues`, `interaction_scores`, ...).
- `OUTDIR/cpdb_results_summary/`  
  - `cpdb_fibro_hepato_sig_counts.tsv`: tidy counts per Fibro–Hepato subtype pair  
  - `cpdb_fibro_hepato_sig_matrix.tsv`: Fibro×Hepato matrix of significant interaction counts

### NicheNet
- `NICHENET_OUT_DIR/`  
  For each receiver `Hepato__<Subtype>`:
  - `receiver_<...>_ligand_activities.tsv`
  - `receiver_<...>_receiver_markers.tsv`
  - `receiver_<...>_avgexpr_top_ligands_in_fibro.tsv`
  - `receiver_<...>_lr_links.tsv`
  - `ALL_receivers_ligand_activities.tsv` (combined)
